# Data Analysis

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

/Users/seandiehl/opt/anaconda3/envs/thesis-env/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df = pd.read_csv('/Users/seandiehl/Desktop/PBP_1819updated1.csv')
df.head()

,X1,GAME_ID,PLAYER1_ID,PLAYER1_NAME,PLAYER2_ID,PLAYER2_NAME,PLAYER3_ID,PLAYER3_NAME,TIME,ASSIST_PLAYER_ID,ASSIST_COUNT,BLOCK_PLAYER_ID,BLOCK_COUNT,REBOUND_PLAYER_ID,REBOUND_TEAM,REBOUND_OFFENSIVE_COUNT,REBOUND_DEFENSIVE_COUNT,FREE_THROW_PLAYER_ID,FREE_THROW_NUM,FREE_THROW_TOTAL,FREE_THROW_MADE,SHOT_PLAYER_ID,POINTS_SCORED,TOTAL_POINTS_SCORED,SHOT_MADE,SHOT_TYPE,STEAL_PLAYER_ID,STEAL_COUNT,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5,HOME_PLAYER_ID_1_PLAY_TIME,HOME_PLAYER_ID_2_PLAY_TIME,HOME_PLAYER_ID_3_PLAY_TIME,HOME_PLAYER_ID_4_PLAY_TIME,HOME_PLAYER_ID_5_PLAY_TIME,AWAY_PLAYER_ID_1_PLAY_TIME,AWAY_PLAYER_ID_2_PLAY_TIME,AWAY_PLAYER_ID_3_PLAY_TIME,AWAY_PLAYER_ID_4_PLAY_TIME,AWAY_PLAYER_ID_5_PLAY_TIME,HOME_PLAYER_ID_1_EXP_PTS,HOME_PLAYER_ID_2_EXP_PTS,HOME_PLAYER_ID_3_EXP_PTS,HOME_PLAYER_ID_4_EXP_PTS,HOME_PLAYER_ID_5_EXP_PTS,AWAY_PLAYER_ID_1_EXP_PTS,AWAY_PLAYER_ID_2_EXP_PTS,AWAY_PLAYER_ID_3_EXP_PTS,AWAY_PLAYER_ID_4_EXP_PTS,AWAY_PLAYER_ID_5_EXP_PTS,HOME_PLAYER_ID_1_EXP_RBS,HOME_PLAYER_ID_2_EXP_RBS,HOME_PLAYER_ID_3_EXP_RBS,HOME_PLAYER_ID_4_EXP_RBS,HOME_PLAYER_ID_5_EXP_RBS,AWAY_PLAYER_ID_1_EXP_RBS,AWAY_PLAYER_ID_2_EXP_RBS,AWAY_PLAYER_ID_3_EXP_RBS,AWAY_PLAYER_ID_4_EXP_RBS,AWAY_PLAYER_ID_5_EXP_RBS,HOME_PLAYER_ID_1_EXP_AST,HOME_PLAYER_ID_2_EXP_AST,HOME_PLAYER_ID_3_EXP_AST,HOME_PLAYER_ID_4_EXP_AST,HOME_PLAYER_ID_5_EXP_AST,AWAY_PLAYER_ID_1_EXP_AST,AWAY_PLAYER_ID_2_EXP_AST,AWAY_PLAYER_ID_3_EXP_AST,AWAY_PLAYER_ID_4_EXP_AST,AWAY_PLAYER_ID_5_EXP_AST,HOME_PLAYER_ID_1_EXP_STLS,HOME_PLAYER_ID_2_EXP_STLS,HOME_PLAYER_ID_3_EXP_STLS,HOME_PLAYER_ID_4_EXP_STLS,HOME_PLAYER_ID_5_EXP_STLS,AWAY_PLAYER_ID_1_EXP_STLS,AWAY_PLAYER_ID_2_EXP_STLS,AWAY_PLAYER_ID_3_EXP_STLS,AWAY_PLAYER_ID_4_EXP_STLS,AWAY_PLAYER_ID_5_EXP_STLS,HOME_PLAYER_ID_1_EXP_BLKS,HOME_PLAYER_ID_2_EXP_BLKS,HOME_PLAYER_ID_3_EXP_BLKS,HOME_PLAYER_ID_4_EXP_BLKS,HOME_PLAYER_ID_5_EXP_BLKS,AWAY_PLAYER_ID_1_EXP_BLKS,AWAY_PLAYER_ID_2_EXP_BLKS,AWAY_PLAYER_ID_3_EXP_BLKS,AWAY_PLAYER_ID_4_EXP_BLKS,AWAY_PLAYER_ID_5_EXP_BLKS
0,0,21801052,0,NaN,0,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!
1,1,21801052,201143,Al Horford,203999,Nikola Jokic,200794,Paul Millsap,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,21801052,200794,Paul Millsap,0,NaN,0,NaN,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200794.0,NaN,NaN,False,Shot,NaN,NaN,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,0.229583333,0.111712963,0.132824074,0.145138889,0.148657407,0.164490741,0.155694444,0.181203704,0.159212963,0.150416667,0.035185185,0.040462963,0.07037037,0.053657407,0.054537037,0.038703704,0.028148148,0.105555556,0.07125,0.050138889,0.050138889,0.04837963,0.044861111,0.018472222,0.018472222,0.033425926,0.027268519,0.056296296,0.030787037,0.037824074,0.010555556,0.014953704,0.007037037,0.007037037,0.011435185,0.009675926,0.014953704,0.011435185,0.012314815,0.009675926,0.003518519,0.004398148,0.011435185,0.002638889,0.007916667,0.003518519,0.001759259,0.007916667,0.010555556,0.006157407
3,3,21801052,1628369,Jayson Tatum,0,NaN,0,NaN,21,NaN,NaN,NaN,NaN,162

The first stat we are going to breakdown is points, because it is the most troublesome. The 'TOTAL_POINTS_SCORED' column does not accurately update the points scored at the free throw line. It does take them into account when the next field goal is scored, but in the interim, the 'TOTAL_POINTS_SCORED' column does not reflect the update. Say, for example, a player entered the game and their first points scored were two free throws, followed by a layup. Their corresponding 'TOTAL_POINTS_SCORED' column would read a value of zero, until they scored their first field goal, at which point it would return four.

Another factor that needs to be taken into account is that substitutions can displace the player_ids. For example, a player_id can shift from 'HOME_PLAYER_ID_1' to 'HOME_PLAYER_ID_2' if a substitution was made. This is no good. It means we can't calculate the actual statistics across each of the player_id columns, because the values can shift. In order to combat this, we are going to create a column for each player_id, that way we can subsequently match the player_ids to the corresponding column. 

I am going to load in a smaller dataframe with only the values relevant for the points statistics, along with each of the player id's.

In [49]:
points = pd.read_csv('/Users/seandiehl/Desktop/actual_points.csv')
points.head()


,GAME_ID,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5,HOME_PLAYER_ID_1_ACT_PTS,HOME_PLAYER_ID_2_ACT_PTS,HOME_PLAYER_ID_3_ACT_PTS,HOME_PLAYER_ID_4_ACT_PTS,AWAY_PLAYER_ID_1_ACT_PTS,AWAY_PLAYER_ID_2_ACT_PTS,AWAY_PLAYER_ID_3_ACT_PTS,AWAY_PLAYER_ID_4_ACT_PTS,AWAY_PLAYER_ID_5_ACT_POINTS,TIME,SHOT_PLAYER_ID,SHOT_MADE,TOTAL_POINTS_SCORED,FREE_THROW_PLAYER_ID,FREE_THROW_NUM,FREE_THROW_MADE,203518,203112,203500,1628389,200746,1628386,1628443,202329,1626147,203937,201583,1628387,203507,2546,1628384,1627853,2772,201571,1628407,1627735,1628366,203084,203115,201587,203382,203145,203078,1627736,201563,1628395,1627761,1627733,202722,201976,203920,1628444,202687,202357,1628469,202339,203992,202711,1626164,1628449,202340,1628396,1627763,1628415,202705,1627759,203485,1628425,1628418,203493,203504,202692,202710,203998,203484,203477,203991,201960,203487,1627936,1626161,1628463,201163,1627737,203546,203903,1628499,1628381,1628380,201954,201144,1626192,1626188,202326,203496,203459,1628470,203109,201967,201939,203584,203076,1627738,202334,203473,1626155,203521,201942,1627767,203476,203915,1628416,1628422,1628408,201609,203083,1627739,201142,1627740,201961,203954,203516,1628393,201936,203957,202702,202324,1626245,1628390,1627812,1627827,1627854,203095,1628368,204025,202683,1628365,201568,204038,201188,200752,202331,201959,203497,203932,201569,1626203,203924,1626170,1627982,201980,203110,203210,201145,201933,1627772,203501,201935,203090,1626149,203914,203925,202699,1626150,1627885,1628404,2617,202330,203089,1627823,1626195,1626209,1628439,1627741,201588,203524,201950,203200,204066,1626178,1626158,203918,203816,201143,1627863,2730,1626154,201586,2738,101141,204060,201281,1627742,202681,1628371,1628411,1627743,1628367,1628382,202704,2544,2229,201973,101161,201949,2207,1626169,204020,202325,203999,1627745,1627884,1627883,1626145,201599,202709,1626163,1628379,203077,1628467,1627788,1628436,201585,1628398,1627746,203087,203897,1627774,1628388,201584,1627814,1627215,203458,202695,203086,1627747,203081,202391,1626172,201572,201577,201567,200768,1627789,1628399,1626168,202714,203705,1627748,1626246,1628374,1626185,1628412,202083,201601,1627775,203468,204456,203926,201580,203585,1628035,203463,201975,1626257,203114,101139,203121,1626259,201988,200794,202703,1628378,1628513,1628370,202328,202734,203961,1628537,202694,202693,1628420,1628405,1626144,1627749,1627750,203488,1627846,1626204,203894,203526,1627777,201149,203457,1628373,203994,1628021,1626220,203124,1628400,1626143,203506,203482,1626224,1626162,203953,202335,1628383,101108,1626166,203901,1627780,1628409,203486,203101,1627751,203490,1626171,204001,203939,1626181,1627816,1627752,1627753,1628397,203944,1628432,1626196,1627781,203085,203460,1628421,203922,200765,201565,203082,1626179,201937,1626156,1627734,203960,203967,203107,203471,203118,1627782,202697,1627783,1627732,203613,1628424,203935,1628372,202397,203503,203917,202362,1628410,1628403,1628369,201952,202066,1628462,1628464,202738,202498,202691,202684,1628414,201229,1626157,200782,1626167,202327,1627755,202685,1627756,1627832,203943,202696,203079,202689,202322,1627820,1628476,203933,201566,1628401,1627855,202355,203952,1626210,203710,101150,1627786,1628391,1626159,1626153,1627824,201152,203469,203092,1627790,1627826
0,21801052,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

The following funuction will be used to generate accurate total points scored for each of the player_ids. 

In [51]:

def apply_points(player_id):
    
    # note 1
    points.loc[(points['TIME']==0), str(player_id)] = 0
    # note 2
    points.loc[((points['SHOT_MADE']==True)&(points['SHOT_PLAYER_ID']==player_id)), str(player_id)] = points['TOTAL_POINTS_SCORED']
    # note 3
    points[str(player_id)] = points[str(player_id)].ffill()
    # note 4
    points.loc[((points['FREE_THROW_PLAYER_ID']==player_id)&(points['FREE_THROW_MADE']==True)&(points['FREE_THROW_NUM']==1)), str(player_id)] += 1
    points[str(player_id)] = points.groupby('GAME_ID')[str(player_id)].cummax()
    points.loc[((points['FREE_THROW_PLAYER_ID']==player_id)&(points['FREE_THROW_MADE']==True)&(points['FREE_THROW_NUM']==2)), str(player_id)] += 1
    points[str(player_id)] = points.groupby('GAME_ID')[str(player_id)].cummax() 
    points.loc[((points['FREE_THROW_PLAYER_ID']==player_id)&(points['FREE_THROW_MADE']==True)&(points['FREE_THROW_NUM']==3)), str(player_id)] += 1
    points[str(player_id)] = points.groupby('GAME_ID')[str(player_id)].cummax()
    #note 5
    points[str(player_id)] = points[str(player_id)].shift(1)
    


Note 1 - this locates all the rows in the dataset where the time is equal to zero, and sets the player's score equal to zero as well, since a player can't have any points at the start of the game. <br>
Note 2 - this locates all the points where the shot is made and the shooting player matches the appropriate player id - then, it updates their the player's column appropriately <br>
Note 3 - this forward fills all the points until the next recorded value. So, starting at the zeros, the player's column will return zero until they make their first shot, when those points will be filled forward until their next shot. <br>
Note 4 - taking the forward filled points, we want to grab the first free throw of a series and add one, then we want to take the maximum value of points and fill that forward for each game. This is then repeated for free throw numbers 2 and 3. <br>
Note 5 - finally, we are going to shift the points column down by one value. because the total points column is updated on the same row the player takes the shot, we need to shift everything down by one row. Otherwise, when we do our final analysis by filtering makes and misses, the "Heat" variable will inaccurately reflect the difference between Actual and Expected points when the shots are taken. <br>

In [52]:
for column in points.columns[27:]:
    player_id = int(column)
    apply_points(player_id)

In [54]:
points.to_csv('/Users/seandiehl/Desktop/actual_points_updated_ids.csv')

Now, I am going to hop over to excel to udpate the column values for the player_id's -- I'm sure there's a more efficient way to do it in python, but I know how to do it in VBA so that's what I'll stick with for this portion. 

In [55]:
points_upd = pd.read_csv('/Users/seandiehl/Desktop/actual_points_updated_ids.csv')

Now that we've loaded it back in, lets add the expected points column to the updated dataframe and investigate.

In [58]:
points_upd[['HOME_PLAYER_ID_1_EXP_PTS', 'HOME_PLAYER_ID_2_EXP_PTS', 'HOME_PLAYER_ID_3_EXP_PTS', 'HOME_PLAYER_ID_4_EXP_PTS', 'HOME_PLAYER_ID_5_EXP_PTS', 'AWAY_PLAYER_ID_1_EXP_PTS', 'AWAY_PLAYER_ID_2_EXP_PTS','AWAY_PLAYER_ID_3_EXP_PTS', 'AWAY_PLAYER_ID_4_EXP_PTS', 'AWAY_PLAYER_ID_5_EXP_PTS']] = df[['HOME_PLAYER_ID_1_EXP_PTS', 'HOME_PLAYER_ID_2_EXP_PTS', 'HOME_PLAYER_ID_3_EXP_PTS', 'HOME_PLAYER_ID_4_EXP_PTS', 'HOME_PLAYER_ID_5_EXP_PTS', 'AWAY_PLAYER_ID_1_EXP_PTS', 'AWAY_PLAYER_ID_2_EXP_PTS','AWAY_PLAYER_ID_3_EXP_PTS', 'AWAY_PLAYER_ID_4_EXP_PTS', 'AWAY_PLAYER_ID_5_EXP_PTS']]
points_upd.head()

,Unnamed: 0,GAME_ID,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5,HOME_PLAYER_ID_1_ACT_PTS,HOME_PLAYER_ID_2_ACT_PTS,HOME_PLAYER_ID_3_ACT_PTS,HOME_PLAYER_ID_4_ACT_PTS,HOME_PLAYER_ID_5_ACT_PTS,AWAY_PLAYER_ID_1_ACT_PTS,AWAY_PLAYER_ID_2_ACT_PTS,AWAY_PLAYER_ID_3_ACT_PTS,AWAY_PLAYER_ID_4_ACT_PTS,AWAY_PLAYER_ID_5_ACT_POINTS,TIME,SHOT_PLAYER_ID,SHOT_MADE,TOTAL_POINTS_SCORED,FREE_THROW_PLAYER_ID,FREE_THROW_NUM,FREE_THROW_MADE,203518,203112,203500,1628389,200746,1628386,1628443,202329,1626147,203937,201583,1628387,203507,2546,1628384,1627853,2772,201571,1628407,1627735,1628366,203084,203115,201587,203382,203145,203078,1627736,201563,1628395,1627761,1627733,202722,201976,203920,1628444,202687,202357,1628469,202339,203992,202711,1626164,1628449,202340,1628396,1627763,1628415,202705,1627759,203485,1628425,1628418,203493,203504,202692,202710,203998,203484,203477,203991,201960,203487,1627936,1626161,1628463,201163,1627737,203546,203903,1628499,1628381,1628380,201954,201144,1626192,1626188,202326,203496,203459,1628470,203109,201967,201939,203584,203076,1627738,202334,203473,1626155,203521,201942,1627767,203476,203915,1628416,1628422,1628408,201609,203083,1627739,201142,1627740,201961,203954,203516,1628393,201936,203957,202702,202324,1626245,1628390,1627812,1627827,1627854,203095,1628368,204025,202683,1628365,201568,204038,201188,200752,202331,201959,203497,203932,201569,1626203,203924,1626170,1627982,201980,203110,203210,201145,201933,1627772,203501,201935,203090,1626149,203914,203925,202699,1626150,1627885,1628404,2617,202330,203089,1627823,1626195,1626209,1628439,1627741,201588,203524,201950,203200,204066,1626178,1626158,203918,203816,201143,1627863,2730,1626154,201586,2738,101141,204060,201281,1627742,202681,1628371,1628411,1627743,1628367,1628382,202704,2544,2229,201973,101161,201949,2207,1626169,204020,202325,203999,1627745,1627884,1627883,1626145,201599,202709,1626163,1628379,203077,1628467,1627788,1628436,201585,1628398,1627746,203087,203897,1627774,1628388,201584,1627814,1627215,203458,202695,203086,1627747,203081,202391,1626172,201572,201577,201567,200768,1627789,1628399,1626168,202714,203705,1627748,1626246,1628374,1626185,1628412,202083,201601,1627775,203468,204456,203926,201580,203585,1628035,203463,201975,1626257,203114,101139,203121,1626259,201988,200794,202703,1628378,1628513,1628370,202328,202734,203961,1628537,202694,202693,1628420,1628405,1626144,1627749,1627750,203488,1627846,1626204,203894,203526,1627777,201149,203457,1628373,203994,1628021,1626220,203124,1628400,1626143,203506,203482,1626224,1626162,203953,202335,1628383,101108,1626166,203901,1627780,1628409,203486,203101,1627751,203490,1626171,204001,203939,1626181,1627816,1627752,1627753,1628397,203944,1628432,1626196,1627781,203085,203460,1628421,203922,200765,201565,203082,1626179,201937,1626156,1627734,203960,203967,203107,203471,203118,1627782,202697,1627783,1627732,203613,1628424,203935,1628372,202397,203503,203917,202362,1628410,1628403,1628369,201952,202066,1628462,1628464,202738,202498,202691,202684,1628414,201229,1626157,200782,1626167,202327,1627755,202685,1627756,1627832,203943,202696,203079,202689,202322,1627820,1628476,203933,201566,1628401,1627855,202355,203952,1626210,203710,101150,1627786,1628391,1626159,1626153,1627824,201152,203469,203092,1627790,1627826,HOME_PLAYER_ID_1_EXP_PTS,HOME_PLAYER_ID_2_EXP_PTS,HOME_PLAYER_ID_3_EXP_PTS,HOME_PLAYER_ID_4_EXP_PTS,HOME_PLAYER_ID_5_EXP_PTS,AWAY_PLAYER_ID_1_EXP_PTS,AWAY_PLAYER_ID_2_EXP_PTS,AWAY_PLAYER_ID_3_EXP_PTS,AWAY_PLAYER_ID_4_EXP_PTS,AWAY_PLAYER_ID_5_EXP_PTS
0,0,21801052,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

The next step is to subtract the actual points from the expected points.

In [70]:
points_upd = points_upd.astype(float)
# points_upd.replace(to_replace='#VALUE!', value=np.nan, inplace=True)
# points_upd.replace(to_replace="NaN", value=np.nan)
points_upd['HOME_PLAYER_ID_1_PTS_DIFF'] = points_upd['HOME_PLAYER_ID_1_ACT_PTS'] - points_upd['HOME_PLAYER_ID_1_EXP_PTS']
points_upd['HOME_PLAYER_ID_2_PTS_DIFF'] = points_upd['HOME_PLAYER_ID_2_ACT_PTS'] - points_upd['HOME_PLAYER_ID_2_EXP_PTS']
points_upd['HOME_PLAYER_ID_3_PTS_DIFF'] = points_upd['HOME_PLAYER_ID_3_ACT_PTS'] - points_upd['HOME_PLAYER_ID_3_EXP_PTS']
points_upd['HOME_PLAYER_ID_4_PTS_DIFF'] = points_upd['HOME_PLAYER_ID_4_ACT_PTS'] - points_upd['HOME_PLAYER_ID_4_EXP_PTS']
points_upd['HOME_PLAYER_ID_5_PTS_DIFF'] = points_upd['HOME_PLAYER_ID_5_ACT_PTS'] - points_upd['HOME_PLAYER_ID_5_EXP_PTS']
points_upd['AWAY_PLAYER_ID_1_PTS_DIFF'] = points_upd['AWAY_PLAYER_ID_1_ACT_PTS'] - points_upd['AWAY_PLAYER_ID_1_EXP_PTS']
points_upd['AWAY_PLAYER_ID_2_PTS_DIFF'] = points_upd['AWAY_PLAYER_ID_2_ACT_PTS'] - points_upd['AWAY_PLAYER_ID_2_EXP_PTS']
points_upd['AWAY_PLAYER_ID_3_PTS_DIFF'] = points_upd['AWAY_PLAYER_ID_3_ACT_PTS'] - points_upd['AWAY_PLAYER_ID_3_EXP_PTS']
points_upd['AWAY_PLAYER_ID_4_PTS_DIFF'] = points_upd['AWAY_PLAYER_ID_4_ACT_PTS'] - points_upd['AWAY_PLAYER_ID_4_EXP_PTS']
points_upd['AWAY_PLAYER_ID_5_PTS_DIFF'] = points_upd['AWAY_PLAYER_ID_5_ACT_POINTS'] - points_upd['AWAY_PLAYER_ID_5_EXP_PTS']




points_upd.head()

,Unnamed: 0,GAME_ID,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5,HOME_PLAYER_ID_1_ACT_PTS,HOME_PLAYER_ID_2_ACT_PTS,HOME_PLAYER_ID_3_ACT_PTS,HOME_PLAYER_ID_4_ACT_PTS,HOME_PLAYER_ID_5_ACT_PTS,AWAY_PLAYER_ID_1_ACT_PTS,AWAY_PLAYER_ID_2_ACT_PTS,AWAY_PLAYER_ID_3_ACT_PTS,AWAY_PLAYER_ID_4_ACT_PTS,AWAY_PLAYER_ID_5_ACT_POINTS,TIME,SHOT_PLAYER_ID,SHOT_MADE,TOTAL_POINTS_SCORED,FREE_THROW_PLAYER_ID,FREE_THROW_NUM,FREE_THROW_MADE,203518,203112,203500,1628389,200746,1628386,1628443,202329,1626147,203937,201583,1628387,203507,2546,1628384,1627853,2772,201571,1628407,1627735,1628366,203084,203115,201587,203382,203145,203078,1627736,201563,1628395,1627761,1627733,202722,201976,203920,1628444,202687,202357,1628469,202339,203992,202711,1626164,1628449,202340,1628396,1627763,1628415,202705,1627759,203485,1628425,1628418,203493,203504,202692,202710,203998,203484,203477,203991,201960,203487,1627936,1626161,1628463,201163,1627737,203546,203903,1628499,1628381,1628380,201954,201144,1626192,1626188,202326,203496,203459,1628470,203109,201967,201939,203584,203076,1627738,202334,203473,1626155,203521,201942,1627767,203476,203915,1628416,1628422,1628408,201609,203083,1627739,201142,1627740,201961,203954,203516,1628393,201936,203957,202702,202324,1626245,1628390,1627812,1627827,1627854,203095,1628368,204025,202683,1628365,201568,204038,201188,200752,202331,201959,203497,203932,201569,1626203,203924,1626170,1627982,201980,203110,203210,201145,201933,1627772,203501,201935,203090,1626149,203914,203925,202699,1626150,1627885,1628404,2617,202330,203089,1627823,1626195,1626209,1628439,1627741,201588,203524,201950,203200,204066,1626178,1626158,203918,203816,201143,1627863,2730,1626154,201586,2738,101141,204060,201281,1627742,202681,1628371,1628411,1627743,1628367,1628382,202704,2544,2229,201973,101161,201949,2207,1626169,204020,202325,203999,1627745,1627884,1627883,1626145,201599,202709,1626163,1628379,203077,1628467,1627788,1628436,201585,1628398,1627746,203087,203897,1627774,1628388,201584,1627814,1627215,203458,202695,203086,1627747,203081,202391,1626172,201572,201577,201567,200768,1627789,1628399,1626168,202714,203705,1627748,1626246,1628374,1626185,1628412,202083,201601,1627775,203468,204456,203926,201580,203585,1628035,203463,201975,1626257,203114,101139,203121,1626259,201988,200794,202703,1628378,1628513,1628370,202328,202734,203961,1628537,202694,202693,1628420,1628405,1626144,1627749,1627750,203488,1627846,1626204,203894,203526,1627777,201149,203457,1628373,203994,1628021,1626220,203124,1628400,1626143,203506,203482,1626224,1626162,203953,202335,1628383,101108,1626166,203901,1627780,1628409,203486,203101,1627751,203490,1626171,204001,203939,1626181,1627816,1627752,1627753,1628397,203944,1628432,1626196,1627781,203085,203460,1628421,203922,200765,201565,203082,1626179,201937,1626156,1627734,203960,203967,203107,203471,203118,1627782,202697,1627783,1627732,203613,1628424,203935,1628372,202397,203503,203917,202362,1628410,1628403,1628369,201952,202066,1628462,1628464,202738,202498,202691,202684,1628414,201229,1626157,200782,1626167,202327,1627755,202685,1627756,1627832,203943,202696,203079,202689,202322,1627820,1628476,203933,201566,1628401,1627855,202355,203952,1626210,203710,101150,1627786,1628391,1626159,1626153,1627824,201152,203469,203092,1627790,1627826,HOME_PLAYER_ID_1_EXP_PTS,HOME_PLAYER_ID_2_EXP_PTS,HOME_PLAYER_ID_3_EXP_PTS,HOME_PLAYER_ID_4_EXP_PTS,HOME_PLAYER_ID_5_EXP_PTS,AWAY_PLAYER_ID_1_EXP_PTS,AWAY_PLAYER_ID_2_EXP_PTS,AWAY_PLAYER_ID_3_EXP_PTS,AWAY_PLAYER_ID_4_EXP_PTS,AWAY_PLAYER_ID_5_EXP_PTS,HOME_PLAYER_ID_1_PTS_DIFF,HOME_PLAYER_ID_2_PTS_DIFF,HOME_PLAYER_ID_3_PTS_DIFF,HOME_PLAYER_ID_4_PTS_DIFF,HOME_PLAYER_ID_5_PTS_DIFF,AWAY_PLAYER_ID_1_PTS_DIFF,AWAY_PLAYER_ID_2_PTS_DIFF,AWAY_PLAYER_ID_3_PTS_DIFF,AWAY_PLAYER_ID_4_PTS_DIFF,AWAY_PLAYER_ID_5_PTS_DIFF
0,0.0,21801052.0,202681.0,203935.0,201143.0,202694.0,1628369.0,1627750.0,2039

Next, we are going to perform a z-score normalization on each of the columns. Because the columns contain many different players, the benefit here is that 

In [113]:
points_upd['HOME_PLAYER_ID_1_PTS_ADJ'] = (points_upd['HOME_PLAYER_ID_1_PTS_DIFF'] - points_upd['HOME_PLAYER_ID_1_PTS_DIFF'].mean()) / points_upd['HOME_PLAYER_ID_1_PTS_DIFF'].std()
points_upd['HOME_PLAYER_ID_2_PTS_ADJ'] = (points_upd['HOME_PLAYER_ID_2_PTS_DIFF'] - points_upd['HOME_PLAYER_ID_2_PTS_DIFF'].mean()) / points_upd['HOME_PLAYER_ID_2_PTS_DIFF'].std()
points_upd['HOME_PLAYER_ID_3_PTS_ADJ'] = (points_upd['HOME_PLAYER_ID_3_PTS_DIFF'] - points_upd['HOME_PLAYER_ID_3_PTS_DIFF'].mean()) / points_upd['HOME_PLAYER_ID_3_PTS_DIFF'].std()
points_upd['HOME_PLAYER_ID_4_PTS_ADJ'] = (points_upd['HOME_PLAYER_ID_4_PTS_DIFF'] - points_upd['HOME_PLAYER_ID_4_PTS_DIFF'].mean()) / points_upd['HOME_PLAYER_ID_4_PTS_DIFF'].std()
points_upd['HOME_PLAYER_ID_5_PTS_ADJ'] = (points_upd['HOME_PLAYER_ID_5_PTS_DIFF'] - points_upd['HOME_PLAYER_ID_5_PTS_DIFF'].mean()) / points_upd['HOME_PLAYER_ID_5_PTS_DIFF'].std()

points_upd['AWAY_PLAYER_ID_1_PTS_ADJ'] = (points_upd['AWAY_PLAYER_ID_1_PTS_DIFF'] - points_upd['AWAY_PLAYER_ID_1_PTS_DIFF'].mean()) / points_upd['AWAY_PLAYER_ID_1_PTS_DIFF'].std()
points_upd['AWAY_PLAYER_ID_2_PTS_ADJ'] = (points_upd['AWAY_PLAYER_ID_2_PTS_DIFF'] - points_upd['AWAY_PLAYER_ID_2_PTS_DIFF'].mean()) / points_upd['AWAY_PLAYER_ID_2_PTS_DIFF'].std()
points_upd['AWAY_PLAYER_ID_3_PTS_ADJ'] = (points_upd['AWAY_PLAYER_ID_3_PTS_DIFF'] - points_upd['AWAY_PLAYER_ID_3_PTS_DIFF'].mean()) / points_upd['AWAY_PLAYER_ID_3_PTS_DIFF'].std()
points_upd['AWAY_PLAYER_ID_4_PTS_ADJ'] = (points_upd['AWAY_PLAYER_ID_4_PTS_DIFF'] - points_upd['AWAY_PLAYER_ID_4_PTS_DIFF'].mean()) / points_upd['AWAY_PLAYER_ID_4_PTS_DIFF'].std()
points_upd['AWAY_PLAYER_ID_5_PTS_ADJ'] = (points_upd['AWAY_PLAYER_ID_5_PTS_DIFF'] - points_upd['AWAY_PLAYER_ID_5_PTS_DIFF'].mean()) / points_upd['AWAY_PLAYER_ID_5_PTS_DIFF'].std()


points_upd.head()

,Unnamed: 0,GAME_ID,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5,HOME_PLAYER_ID_1_ACT_PTS,HOME_PLAYER_ID_2_ACT_PTS,HOME_PLAYER_ID_3_ACT_PTS,HOME_PLAYER_ID_4_ACT_PTS,HOME_PLAYER_ID_5_ACT_PTS,AWAY_PLAYER_ID_1_ACT_PTS,AWAY_PLAYER_ID_2_ACT_PTS,AWAY_PLAYER_ID_3_ACT_PTS,AWAY_PLAYER_ID_4_ACT_PTS,AWAY_PLAYER_ID_5_ACT_POINTS,TIME,SHOT_PLAYER_ID,SHOT_MADE,TOTAL_POINTS_SCORED,FREE_THROW_PLAYER_ID,FREE_THROW_NUM,FREE_THROW_MADE,203518,203112,203500,1628389,200746,1628386,1628443,202329,1626147,203937,201583,1628387,203507,2546,1628384,1627853,2772,201571,1628407,1627735,1628366,203084,203115,201587,203382,203145,203078,1627736,201563,1628395,1627761,1627733,202722,201976,203920,1628444,202687,202357,1628469,202339,203992,202711,1626164,1628449,202340,1628396,1627763,1628415,202705,1627759,203485,1628425,1628418,203493,203504,202692,202710,203998,203484,203477,203991,201960,203487,1627936,1626161,1628463,201163,1627737,203546,203903,1628499,1628381,1628380,201954,201144,1626192,1626188,202326,203496,203459,1628470,203109,201967,201939,203584,203076,1627738,202334,203473,1626155,203521,201942,1627767,203476,203915,1628416,1628422,1628408,201609,203083,1627739,201142,1627740,201961,203954,203516,1628393,201936,203957,202702,202324,1626245,1628390,1627812,1627827,1627854,203095,1628368,204025,202683,1628365,201568,204038,201188,200752,202331,201959,203497,203932,201569,1626203,203924,1626170,1627982,201980,203110,203210,201145,201933,1627772,203501,201935,203090,1626149,203914,203925,202699,1626150,1627885,1628404,2617,202330,203089,1627823,1626195,1626209,1628439,1627741,201588,203524,201950,203200,204066,1626178,1626158,203918,203816,201143,1627863,2730,1626154,201586,2738,101141,204060,201281,1627742,202681,1628371,1628411,1627743,1628367,1628382,202704,2544,2229,201973,101161,201949,2207,1626169,204020,202325,203999,1627745,1627884,1627883,1626145,201599,202709,1626163,1628379,203077,1628467,1627788,1628436,201585,1628398,1627746,203087,203897,1627774,1628388,201584,1627814,1627215,203458,202695,203086,1627747,203081,202391,1626172,201572,201577,201567,200768,1627789,1628399,1626168,202714,203705,1627748,1626246,1628374,1626185,1628412,202083,201601,1627775,203468,204456,203926,201580,203585,1628035,203463,201975,1626257,203114,101139,203121,1626259,201988,200794,202703,1628378,1628513,1628370,202328,202734,203961,1628537,202694,202693,1628420,1628405,1626144,1627749,1627750,203488,1627846,1626204,203894,203526,1627777,201149,203457,1628373,203994,1628021,1626220,203124,1628400,1626143,203506,203482,1626224,1626162,203953,202335,1628383,101108,1626166,203901,1627780,1628409,203486,203101,1627751,203490,1626171,204001,203939,1626181,1627816,1627752,1627753,1628397,203944,1628432,1626196,1627781,203085,203460,1628421,203922,200765,201565,203082,1626179,201937,1626156,1627734,203960,203967,203107,203471,203118,1627782,202697,1627783,1627732,203613,1628424,203935,1628372,202397,203503,203917,202362,1628410,1628403,1628369,201952,202066,1628462,1628464,202738,202498,202691,202684,1628414,201229,1626157,200782,1626167,202327,1627755,202685,1627756,1627832,203943,202696,203079,202689,202322,1627820,1628476,203933,201566,1628401,1627855,202355,203952,1626210,203710,101150,1627786,1628391,1626159,1626153,1627824,201152,203469,203092,1627790,1627826,HOME_PLAYER_ID_1_EXP_PTS,HOME_PLAYER_ID_2_EXP_PTS,HOME_PLAYER_ID_3_EXP_PTS,HOME_PLAYER_ID_4_EXP_PTS,HOME_PLAYER_ID_5_EXP_PTS,AWAY_PLAYER_ID_1_EXP_PTS,AWAY_PLAYER_ID_2_EXP_PTS,AWAY_PLAYER_ID_3_EXP_PTS,AWAY_PLAYER_ID_4_EXP_PTS,AWAY_PLAYER_ID_5_EXP_PTS,HOME_PLAYER_ID_1_PTS_DIFF,HOME_PLAYER_ID_2_PTS_DIFF,HOME_PLAYER_ID_3_PTS_DIFF,HOME_PLAYER_ID_4_PTS_DIFF,HOME_PLAYER_ID_5_PTS_DIFF,AWAY_PLAYER_ID_1_PTS_DIFF,AWAY_PLAYER_ID_2_PTS_DIFF,AWAY_PLAYER_ID_3_PTS_DIFF,AWAY_PLAYER_ID_4_PTS_DIFF,AWAY_PLAYER_ID_5_PTS_DIFF,HOME_PLAYER_ID_1_PTS_ADJ,HOME_PLAYER_ID_2_PTS_ADJ,HOME_PLAYER_ID_3_PTS_ADJ,HO

In [74]:
rebounds = pd.DataFrame()
rebounds[['HOME_PLAYER_ID_1_EXP_RBS',	'HOME_PLAYER_ID_2_EXP_RBS',	'HOME_PLAYER_ID_3_EXP_RBS',	'HOME_PLAYER_ID_4_EXP_RBS',	'HOME_PLAYER_ID_5_EXP_RBS',	'AWAY_PLAYER_ID_1_EXP_RBS',	'AWAY_PLAYER_ID_2_EXP_RBS',	'AWAY_PLAYER_ID_3_EXP_RBS',	'AWAY_PLAYER_ID_4_EXP_RBS',	'AWAY_PLAYER_ID_5_EXP_RBS']] = df[['HOME_PLAYER_ID_1_EXP_RBS',	'HOME_PLAYER_ID_2_EXP_RBS',	'HOME_PLAYER_ID_3_EXP_RBS',	'HOME_PLAYER_ID_4_EXP_RBS',	'HOME_PLAYER_ID_5_EXP_RBS',	'AWAY_PLAYER_ID_1_EXP_RBS',	'AWAY_PLAYER_ID_2_EXP_RBS',	'AWAY_PLAYER_ID_3_EXP_RBS',	'AWAY_PLAYER_ID_4_EXP_RBS',	'AWAY_PLAYER_ID_5_EXP_RBS']]

In [75]:
rebounds.head()

,HOME_PLAYER_ID_1_EXP_RBS,HOME_PLAYER_ID_2_EXP_RBS,HOME_PLAYER_ID_3_EXP_RBS,HOME_PLAYER_ID_4_EXP_RBS,HOME_PLAYER_ID_5_EXP_RBS,AWAY_PLAYER_ID_1_EXP_RBS,AWAY_PLAYER_ID_2_EXP_RBS,AWAY_PLAYER_ID_3_EXP_RBS,AWAY_PLAYER_ID_4_EXP_RBS,AWAY_PLAYER_ID_5_EXP_RBS
0,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!
1,0,0,0,0,0,0,0,0,0,0
2,0.035185185,0.040462963,0.07037037,0.053657407,0.054537037,0.038703704,0.028148148,0.105555556,0.07125,0.050138889
3,0.038888889,0.044722222,0.077777778,0.059305556,0.060277778,0.042777778,0.031111111,0.116666667,0.07875,0.055416667
4,0.048148148,0.05537037,0.096296296,0.073425926,0.07462963,0.052962963,0.038518519,0.144444444,0.0975,0.068611111


In [76]:
rebounds[['REBOUND_PLAYER_ID',	'REBOUND_TEAM',	'REBOUND_OFFENSIVE_COUNT',	'REBOUND_DEFENSIVE_COUNT']] = df[['REBOUND_PLAYER_ID',	'REBOUND_TEAM',	'REBOUND_OFFENSIVE_COUNT',	'REBOUND_DEFENSIVE_COUNT']]

In [77]:
rebounds.head()

,HOME_PLAYER_ID_1_EXP_RBS,HOME_PLAYER_ID_2_EXP_RBS,HOME_PLAYER_ID_3_EXP_RBS,HOME_PLAYER_ID_4_EXP_RBS,HOME_PLAYER_ID_5_EXP_RBS,AWAY_PLAYER_ID_1_EXP_RBS,AWAY_PLAYER_ID_2_EXP_RBS,AWAY_PLAYER_ID_3_EXP_RBS,AWAY_PLAYER_ID_4_EXP_RBS,AWAY_PLAYER_ID_5_EXP_RBS,REBOUND_PLAYER_ID,REBOUND_TEAM,REBOUND_OFFENSIVE_COUNT,REBOUND_DEFENSIVE_COUNT
0,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,NaN,NaN,NaN,NaN
1,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN
2,0.035185185,0.040462963,0.07037037,0.053657407,0.054537037,0.038703704,0.028148148,0.105555556,0.07125,0.050138889,NaN,NaN,NaN,NaN
3,0.038888889,0.044722222,0.077777778,0.059305556,0.060277778,0.042777778,0.031111111,0.116666667,0.07875,0.055416667,1628369.0,NaN,0.0,1.0
4,0.048148148,0.05537037,0.096296296,0.073425926,0.07462963,0.052962963,0.038518519,0.144444444,0.0975,0.068611111,NaN,NaN,NaN,NaN


In [79]:
rebounds.to_csv('/Users/seandiehl/Desktop/rebounds.csv')

In [81]:
rebounds_upd = pd.read_csv('/Users/seandiehl/Desktop/rebounds_upd.csv')

In [82]:
rebounds_upd.head()

,Unnamed: 0,HOME_PLAYER_ID_1_EXP_RBS,HOME_PLAYER_ID_2_EXP_RBS,HOME_PLAYER_ID_3_EXP_RBS,HOME_PLAYER_ID_4_EXP_RBS,HOME_PLAYER_ID_5_EXP_RBS,AWAY_PLAYER_ID_1_EXP_RBS,AWAY_PLAYER_ID_2_EXP_RBS,AWAY_PLAYER_ID_3_EXP_RBS,AWAY_PLAYER_ID_4_EXP_RBS,AWAY_PLAYER_ID_5_EXP_RBS,REBOUND_PLAYER_ID,REBOUND_TEAM,REBOUND_OFFENSIVE_COUNT,REBOUND_DEFENSIVE_COUNT,203518,203112,203500,1628389,200746,1628386,1628443,202329,1626147,203937,201583,1628387,203507,2546,1628384,1627853,2772,201571,1628407,1627735,1628366,203084,203115,201587,203382,203145,203078,1627736,201563,1628395,1627761,1627733,202722,201976,203920,1628444,202687,202357,1628469,202339,203992,202711,1626164,1628449,202340,1628396,1627763,1628415,202705,1627759,203485,1628425,1628418,203493,203504,202692,202710,203998,203484,203477,203991,201960,203487,1627936,1626161,1628463,201163,1627737,203546,203903,1628499,1628381,1628380,201954,201144,1626192,1626188,202326,203496,203459,1628470,203109,201967,201939,203584,203076,1627738,202334,203473,1626155,203521,201942,1627767,203476,203915,1628416,1628422,1628408,201609,203083,1627739,201142,1627740,201961,203954,203516,1628393,201936,203957,202702,202324,1626245,1628390,1627812,1627827,1627854,203095,1628368,204025,202683,1628365,201568,204038,201188,200752,202331,201959,203497,203932,201569,1626203,203924,1626170,1627982,201980,203110,203210,201145,201933,1627772,203501,201935,203090,1626149,203914,203925,202699,1626150,1627885,1628404,2617,202330,203089,1627823,1626195,1626209,1628439,1627741,201588,203524,201950,203200,204066,1626178,1626158,203918,203816,201143,1627863,2730,1626154,201586,2738,101141,204060,201281,1627742,202681,1628371,1628411,1627743,1628367,1628382,202704,2544,2229,201973,101161,201949,2207,1626169,204020,202325,203999,1627745,1627884,1627883,1626145,201599,202709,1626163,1628379,203077,1628467,1627788,1628436,201585,1628398,1627746,203087,203897,1627774,1628388,201584,1627814,1627215,203458,202695,203086,1627747,203081,202391,1626172,201572,201577,201567,200768,1627789,1628399,1626168,202714,203705,1627748,1626246,1628374,1626185,1628412,202083,201601,1627775,203468,204456,203926,201580,203585,1628035,203463,201975,1626257,203114,101139,203121,1626259,201988,200794,202703,1628378,1628513,1628370,202328,202734,203961,1628537,202694,202693,1628420,1628405,1626144,1627749,1627750,203488,1627846,1626204,203894,203526,1627777,201149,203457,1628373,203994,1628021,1626220,203124,1628400,1626143,203506,203482,1626224,1626162,203953,202335,1628383,101108,1626166,203901,1627780,1628409,203486,203101,1627751,203490,1626171,204001,203939,1626181,1627816,1627752,1627753,1628397,203944,1628432,1626196,1627781,203085,203460,1628421,203922,200765,201565,203082,1626179,201937,1626156,1627734,203960,203967,203107,203471,203118,1627782,202697,1627783,1627732,203613,1628424,203935,1628372,202397,203503,203917,202362,1628410,1628403,1628369,201952,202066,1628462,1628464,202738,202498,202691,202684,1628414,201229,1626157,200782,1626167,202327,1627755,202685,1627756,1627832,203943,202696,203079,202689,202322,1627820,1628476,203933,201566,1628401,1627855,202355,203952,1626210,203710,101150,1627786,1628391,1626159,1626153,1627824,201152,203469,203092,1627790,1627826
0,0,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [92]:
rebounds_upd['TOTAL_RBS'] = rebounds_upd['REBOUND_OFFENSIVE_COUNT'] + rebounds_upd['REBOUND_DEFENSIVE_COUNT']
rebounds_upd['GAME_ID'] = df['GAME_ID']
rebounds_upd['TIME'] = df['TIME']

for column in rebounds_upd.columns[15:-3]:
    print(column)

203518
203112
203500
1628389
200746
1628386
1628443
202329
1626147
203937
201583
1628387
203507
2546
1628384
1627853
2772
201571
1628407
1627735
1628366
203084
203115
201587
203382
203145
203078
1627736
201563
1628395
1627761
1627733
202722
201976
203920
1628444
202687
202357
1628469
202339
203992
202711
1626164
1628449
202340
1628396
1627763
1628415
202705
1627759
203485
1628425
1628418
203493
203504
202692
202710
203998
203484
203477
203991
201960
203487
1627936
1626161
1628463
201163
1627737
203546
203903
1628499
1628381
1628380
201954
201144
1626192
1626188
202326
203496
203459
1628470
203109
201967
201939
203584
203076
1627738
202334
203473
1626155
203521
201942
1627767
203476
203915
1628416
1628422
1628408
201609
203083
1627739
201142
1627740
201961
203954
203516
1628393
201936
203957
202702
202324
1626245
1628390
1627812
1627827
1627854
203095
1628368
204025
202683
1628365
201568
204038
201188
200752
202331
201959
203497
203932
201569
1626203
203924
1626170
1627982
201980
203110

In [95]:

def apply_rebounds(player_id):
    
    # note 1
    rebounds_upd.loc[(rebounds_upd['TIME']==0), str(player_id)] = 0
    # note 2
    # rebounds_upd.loc[((points['SHOT_MADE']==True)&(points['SHOT_PLAYER_ID']==player_id)), str(player_id)] = points['TOTAL_POINTS_SCORED']
    rebounds_upd.loc[(rebounds_upd['REBOUND_PLAYER_ID']==player_id), str(player_id)] = rebounds_upd['TOTAL_RBS']
    rebounds_upd[str(player_id)] = rebounds_upd[str(player_id)].ffill()


    # note 3
    # points[str(player_id)] = points[str(player_id)].ffill()
    # # note 4
    # points.loc[((points['FREE_THROW_PLAYER_ID']==player_id)&(points['FREE_THROW_MADE']==True)&(points['FREE_THROW_NUM']==1)), str(player_id)] += 1
    # points[str(player_id)] = points.groupby('GAME_ID')[str(player_id)].cummax()
    # points.loc[((points['FREE_THROW_PLAYER_ID']==player_id)&(points['FREE_THROW_MADE']==True)&(points['FREE_THROW_NUM']==2)), str(player_id)] += 1
    # points[str(player_id)] = points.groupby('GAME_ID')[str(player_id)].cummax() 
    # points.loc[((points['FREE_THROW_PLAYER_ID']==player_id)&(points['FREE_THROW_MADE']==True)&(points['FREE_THROW_NUM']==3)), str(player_id)] += 1
    # points[str(player_id)] = points.groupby('GAME_ID')[str(player_id)].cummax()
    #note 5
    # points[str(player_id)] = points[str(player_id)].shift(1)




In [96]:
for column in rebounds_upd.columns[15:-3]:
    player_id = int(column)
    apply_rebounds(player_id)

In [ ]:
rebounds_upd[[]]

In [98]:
rebounds_upd.to_csv('/Users/seandiehl/Desktop/rebounds_upd_players.csv')

In [99]:
rebounds_upd[['HOME_PLAYER_ID_1',	'HOME_PLAYER_ID_2',	'HOME_PLAYER_ID_3',	'HOME_PLAYER_ID_4',	'HOME_PLAYER_ID_5',	'AWAY_PLAYER_ID_1',	'AWAY_PLAYER_ID_2',	'AWAY_PLAYER_ID_3',	'AWAY_PLAYER_ID_4',	'AWAY_PLAYER_ID_5']] = df[['HOME_PLAYER_ID_1',	'HOME_PLAYER_ID_2',	'HOME_PLAYER_ID_3',	'HOME_PLAYER_ID_4',	'HOME_PLAYER_ID_5',	'AWAY_PLAYER_ID_1',	'AWAY_PLAYER_ID_2',	'AWAY_PLAYER_ID_3',	'AWAY_PLAYER_ID_4',	'AWAY_PLAYER_ID_5']]

/var/folders/zm/h6ndcwpn4h72pqxjr10sfd680000gn/T/ipykernel_69232/1498560826.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  rebounds_upd[['HOME_PLAYER_ID_1',	'HOME_PLAYER_ID_2',	'HOME_PLAYER_ID_3',	'HOME_PLAYER_ID_4',	'HOME_PLAYER_ID_5',	'AWAY_PLAYER_ID_1',	'AWAY_PLAYER_ID_2',	'AWAY_PLAYER_ID_3',	'AWAY_PLAYER_ID_4',	'AWAY_PLAYER_ID_5']] = df[['HOME_PLAYER_ID_1',	'HOME_PLAYER_ID_2',	'HOME_PLAYER_ID_3',	'HOME_PLAYER_ID_4',	'HOME_PLAYER_ID_5',	'AWAY_PLAYER_ID_1',	'AWAY_PLAYER_ID_2',	'AWAY_PLAYER_ID_3',	'AWAY_PLAYER_ID_4',	'AWAY_PLAYER_ID_5']]
/var/folders/zm/h6ndcwpn4h72pqxjr10sfd680000gn/T/ipykernel_69232/1498560826.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor pe

In [100]:
rebounds_upd.head()

,Unnamed: 0,HOME_PLAYER_ID_1_EXP_RBS,HOME_PLAYER_ID_2_EXP_RBS,HOME_PLAYER_ID_3_EXP_RBS,HOME_PLAYER_ID_4_EXP_RBS,HOME_PLAYER_ID_5_EXP_RBS,AWAY_PLAYER_ID_1_EXP_RBS,AWAY_PLAYER_ID_2_EXP_RBS,AWAY_PLAYER_ID_3_EXP_RBS,AWAY_PLAYER_ID_4_EXP_RBS,AWAY_PLAYER_ID_5_EXP_RBS,REBOUND_PLAYER_ID,REBOUND_TEAM,REBOUND_OFFENSIVE_COUNT,REBOUND_DEFENSIVE_COUNT,203518,203112,203500,1628389,200746,1628386,1628443,202329,1626147,203937,201583,1628387,203507,2546,1628384,1627853,2772,201571,1628407,1627735,1628366,203084,203115,201587,203382,203145,203078,1627736,201563,1628395,1627761,1627733,202722,201976,203920,1628444,202687,202357,1628469,202339,203992,202711,1626164,1628449,202340,1628396,1627763,1628415,202705,1627759,203485,1628425,1628418,203493,203504,202692,202710,203998,203484,203477,203991,201960,203487,1627936,1626161,1628463,201163,1627737,203546,203903,1628499,1628381,1628380,201954,201144,1626192,1626188,202326,203496,203459,1628470,203109,201967,201939,203584,203076,1627738,202334,203473,1626155,203521,201942,1627767,203476,203915,1628416,1628422,1628408,201609,203083,1627739,201142,1627740,201961,203954,203516,1628393,201936,203957,202702,202324,1626245,1628390,1627812,1627827,1627854,203095,1628368,204025,202683,1628365,201568,204038,201188,200752,202331,201959,203497,203932,201569,1626203,203924,1626170,1627982,201980,203110,203210,201145,201933,1627772,203501,201935,203090,1626149,203914,203925,202699,1626150,1627885,1628404,2617,202330,203089,1627823,1626195,1626209,1628439,1627741,201588,203524,201950,203200,204066,1626178,1626158,203918,203816,201143,1627863,2730,1626154,201586,2738,101141,204060,201281,1627742,202681,1628371,1628411,1627743,1628367,1628382,202704,2544,2229,201973,101161,201949,2207,1626169,204020,202325,203999,1627745,1627884,1627883,1626145,201599,202709,1626163,1628379,203077,1628467,1627788,1628436,201585,1628398,1627746,203087,203897,1627774,1628388,201584,1627814,1627215,203458,202695,203086,1627747,203081,202391,1626172,201572,201577,201567,200768,1627789,1628399,1626168,202714,203705,1627748,1626246,1628374,1626185,1628412,202083,201601,1627775,203468,204456,203926,201580,203585,1628035,203463,201975,1626257,203114,101139,203121,1626259,201988,200794,202703,1628378,1628513,1628370,202328,202734,203961,1628537,202694,202693,1628420,1628405,1626144,1627749,1627750,203488,1627846,1626204,203894,203526,1627777,201149,203457,1628373,203994,1628021,1626220,203124,1628400,1626143,203506,203482,1626224,1626162,203953,202335,1628383,101108,1626166,203901,1627780,1628409,203486,203101,1627751,203490,1626171,204001,203939,1626181,1627816,1627752,1627753,1628397,203944,1628432,1626196,1627781,203085,203460,1628421,203922,200765,201565,203082,1626179,201937,1626156,1627734,203960,203967,203107,203471,203118,1627782,202697,1627783,1627732,203613,1628424,203935,1628372,202397,203503,203917,202362,1628410,1628403,1628369,201952,202066,1628462,1628464,202738,202498,202691,202684,1628414,201229,1626157,200782,1626167,202327,1627755,202685,1627756,1627832,203943,202696,203079,202689,202322,1627820,1628476,203933,201566,1628401,1627855,202355,203952,1626210,203710,101150,1627786,1628391,1626159,1626153,1627824,201152,203469,203092,1627790,1627826,TOTAL_RBS,GAME_ID,TIME,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5
0,0,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [103]:
rebounds_upd = rebounds_upd.replace(to_replace="#VALUE!", value=np.nan)
rebounds_upd = rebounds_upd.astype(float)

In [104]:
rebounds_upd.to_csv('/Users/seandiehl/Desktop/rebounds_upd_players_1.csv')

In [105]:
rebounds_upd_1 = pd.read_csv('/Users/seandiehl/Desktop/rebounds_upd_players_1.csv')

In [106]:
rebounds_upd_1.head()

,Unnamed: 0.1,Unnamed: 0,HOME_PLAYER_ID_1_EXP_RBS,HOME_PLAYER_ID_2_EXP_RBS,HOME_PLAYER_ID_3_EXP_RBS,HOME_PLAYER_ID_4_EXP_RBS,HOME_PLAYER_ID_5_EXP_RBS,AWAY_PLAYER_ID_1_EXP_RBS,AWAY_PLAYER_ID_2_EXP_RBS,AWAY_PLAYER_ID_3_EXP_RBS,AWAY_PLAYER_ID_4_EXP_RBS,AWAY_PLAYER_ID_5_EXP_RBS,REBOUND_PLAYER_ID,REBOUND_TEAM,REBOUND_OFFENSIVE_COUNT,REBOUND_DEFENSIVE_COUNT,203518,203112,203500,1628389,200746,1628386,1628443,202329,1626147,203937,201583,1628387,203507,2546,1628384,1627853,2772,201571,1628407,1627735,1628366,203084,203115,201587,203382,203145,203078,1627736,201563,1628395,1627761,1627733,202722,201976,203920,1628444,202687,202357,1628469,202339,203992,202711,1626164,1628449,202340,1628396,1627763,1628415,202705,1627759,203485,1628425,1628418,203493,203504,202692,202710,203998,203484,203477,203991,201960,203487,1627936,1626161,1628463,201163,1627737,203546,203903,1628499,1628381,1628380,201954,201144,1626192,1626188,202326,203496,203459,1628470,203109,201967,201939,203584,203076,1627738,202334,203473,1626155,203521,201942,1627767,203476,203915,1628416,1628422,1628408,201609,203083,1627739,201142,1627740,201961,203954,203516,1628393,201936,203957,202702,202324,1626245,1628390,1627812,1627827,1627854,203095,1628368,204025,202683,1628365,201568,204038,201188,200752,202331,201959,203497,203932,201569,1626203,203924,1626170,1627982,201980,203110,203210,201145,201933,1627772,203501,201935,203090,1626149,203914,203925,202699,1626150,1627885,1628404,2617,202330,203089,1627823,1626195,1626209,1628439,1627741,201588,203524,201950,203200,204066,1626178,1626158,203918,203816,201143,1627863,2730,1626154,201586,2738,101141,204060,201281,1627742,202681,1628371,1628411,1627743,1628367,1628382,202704,2544,2229,201973,101161,201949,2207,1626169,204020,202325,203999,1627745,1627884,1627883,1626145,201599,202709,1626163,1628379,203077,1628467,1627788,1628436,201585,1628398,1627746,203087,203897,1627774,1628388,201584,1627814,1627215,203458,202695,203086,1627747,203081,202391,1626172,201572,201577,201567,200768,1627789,1628399,1626168,202714,203705,1627748,1626246,1628374,1626185,1628412,202083,201601,1627775,203468,204456,203926,201580,203585,1628035,203463,201975,1626257,203114,101139,203121,1626259,201988,200794,202703,1628378,1628513,1628370,202328,202734,203961,1628537,202694,202693,1628420,1628405,1626144,1627749,1627750,203488,1627846,1626204,203894,203526,1627777,201149,203457,1628373,203994,1628021,1626220,203124,1628400,1626143,203506,203482,1626224,1626162,203953,202335,1628383,101108,1626166,203901,1627780,1628409,203486,203101,1627751,203490,1626171,204001,203939,1626181,1627816,1627752,1627753,1628397,203944,1628432,1626196,1627781,203085,203460,1628421,203922,200765,201565,203082,1626179,201937,1626156,1627734,203960,203967,203107,203471,203118,1627782,202697,1627783,1627732,203613,1628424,203935,1628372,202397,203503,203917,202362,1628410,1628403,1628369,201952,202066,1628462,1628464,202738,202498,202691,202684,1628414,201229,1626157,200782,1626167,202327,1627755,202685,1627756,1627832,203943,202696,203079,202689,202322,1627820,1628476,203933,201566,1628401,1627855,202355,203952,1626210,203710,101150,1627786,1628391,1626159,1626153,1627824,201152,203469,203092,1627790,1627826,TOTAL_RBS,GAME_ID,TIME,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5,HOME_PLAYER_ID_1_ACT_RBS,HOME_PLAYER_ID_2_ACT_RBS,HOME_PLAYER_ID_3_ACT_RBS,HOME_PLAYER_ID_4_ACT_RBS,HOME_PLAYER_ID_5_ACT_RBS,AWAY_PLAYER_ID_1_ACT_RBS,AWAY_PLAYER_ID_2_ACT_RBS,AWAY_PLAYER_ID_3_ACT_RBS,AWAY_PLAYER_ID_4_ACT_RBS,AWAY_PLAYER_ID_5_ACT_RBS
0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [107]:
rebounds_upd_1['HOME_PLAYER_ID_1_RB_DIFF'] = rebounds_upd_1['HOME_PLAYER_ID_1_ACT_RBS'] = rebounds_upd_1['HOME_PLAYER_ID_1_EXP_RBS']
rebounds_upd_1['HOME_PLAYER_ID_2_RB_DIFF'] = rebounds_upd_1['HOME_PLAYER_ID_2_ACT_RBS'] = rebounds_upd_1['HOME_PLAYER_ID_2_EXP_RBS']
rebounds_upd_1['HOME_PLAYER_ID_3_RB_DIFF'] = rebounds_upd_1['HOME_PLAYER_ID_3_ACT_RBS'] = rebounds_upd_1['HOME_PLAYER_ID_3_EXP_RBS']
rebounds_upd_1['HOME_PLAYER_ID_4_RB_DIFF'] = rebounds_upd_1['HOME_PLAYER_ID_4_ACT_RBS'] = rebounds_upd_1['HOME_PLAYER_ID_4_EXP_RBS']
rebounds_upd_1['HOME_PLAYER_ID_5_RB_DIFF'] = rebounds_upd_1['HOME_PLAYER_ID_5_ACT_RBS'] = rebounds_upd_1['HOME_PLAYER_ID_5_EXP_RBS']

rebounds_upd_1['AWAY_PLAYER_ID_1_RB_DIFF'] = rebounds_upd_1['AWAY_PLAYER_ID_1_ACT_RBS'] = rebounds_upd_1['AWAY_PLAYER_ID_1_EXP_RBS']
rebounds_upd_1['AWAY_PLAYER_ID_2_RB_DIFF'] = rebounds_upd_1['AWAY_PLAYER_ID_2_ACT_RBS'] = rebounds_upd_1['AWAY_PLAYER_ID_2_EXP_RBS']
rebounds_upd_1['AWAY_PLAYER_ID_3_RB_DIFF'] = rebounds_upd_1['AWAY_PLAYER_ID_3_ACT_RBS'] = rebounds_upd_1['AWAY_PLAYER_ID_3_EXP_RBS']
rebounds_upd_1['AWAY_PLAYER_ID_4_RB_DIFF'] = rebounds_upd_1['AWAY_PLAYER_ID_4_ACT_RBS'] = rebounds_upd_1['AWAY_PLAYER_ID_4_EXP_RBS']
rebounds_upd_1['AWAY_PLAYER_ID_5_RB_DIFF'] = rebounds_upd_1['AWAY_PLAYER_ID_5_ACT_RBS'] = rebounds_upd_1['AWAY_PLAYER_ID_5_EXP_RBS']




In [108]:
rebounds_upd_1['HOME_PLAYER_ID_1_RB_ADJ'] = (rebounds_upd_1['HOME_PLAYER_ID_1_RB_DIFF'] - rebounds_upd_1['HOME_PLAYER_ID_1_RB_DIFF'].mean()) / rebounds_upd_1['HOME_PLAYER_ID_1_RB_DIFF'].std()
rebounds_upd_1['HOME_PLAYER_ID_2_RB_ADJ'] = (rebounds_upd_1['HOME_PLAYER_ID_2_RB_DIFF'] - rebounds_upd_1['HOME_PLAYER_ID_2_RB_DIFF'].mean()) / rebounds_upd_1['HOME_PLAYER_ID_2_RB_DIFF'].std()
rebounds_upd_1['HOME_PLAYER_ID_3_RB_ADJ'] = (rebounds_upd_1['HOME_PLAYER_ID_3_RB_DIFF'] - rebounds_upd_1['HOME_PLAYER_ID_3_RB_DIFF'].mean()) / rebounds_upd_1['HOME_PLAYER_ID_3_RB_DIFF'].std()
rebounds_upd_1['HOME_PLAYER_ID_4_RB_ADJ'] = (rebounds_upd_1['HOME_PLAYER_ID_4_RB_DIFF'] - rebounds_upd_1['HOME_PLAYER_ID_4_RB_DIFF'].mean()) / rebounds_upd_1['HOME_PLAYER_ID_4_RB_DIFF'].std()
rebounds_upd_1['HOME_PLAYER_ID_5_RB_ADJ'] = (rebounds_upd_1['HOME_PLAYER_ID_5_RB_DIFF'] - rebounds_upd_1['HOME_PLAYER_ID_5_RB_DIFF'].mean()) / rebounds_upd_1['HOME_PLAYER_ID_5_RB_DIFF'].std()

rebounds_upd_1['AWAY_PLAYER_ID_1_RB_ADJ'] = (rebounds_upd_1['AWAY_PLAYER_ID_1_RB_DIFF'] - rebounds_upd_1['AWAY_PLAYER_ID_1_RB_DIFF'].mean()) / rebounds_upd_1['AWAY_PLAYER_ID_1_RB_DIFF'].std()
rebounds_upd_1['AWAY_PLAYER_ID_2_RB_ADJ'] = (rebounds_upd_1['AWAY_PLAYER_ID_2_RB_DIFF'] - rebounds_upd_1['AWAY_PLAYER_ID_2_RB_DIFF'].mean()) / rebounds_upd_1['AWAY_PLAYER_ID_2_RB_DIFF'].std()
rebounds_upd_1['AWAY_PLAYER_ID_3_RB_ADJ'] = (rebounds_upd_1['AWAY_PLAYER_ID_3_RB_DIFF'] - rebounds_upd_1['AWAY_PLAYER_ID_3_RB_DIFF'].mean()) / rebounds_upd_1['AWAY_PLAYER_ID_3_RB_DIFF'].std()
rebounds_upd_1['AWAY_PLAYER_ID_4_RB_ADJ'] = (rebounds_upd_1['AWAY_PLAYER_ID_4_RB_DIFF'] - rebounds_upd_1['AWAY_PLAYER_ID_4_RB_DIFF'].mean()) / rebounds_upd_1['AWAY_PLAYER_ID_4_RB_DIFF'].std()
rebounds_upd_1['AWAY_PLAYER_ID_5_RB_ADJ'] = (rebounds_upd_1['AWAY_PLAYER_ID_5_RB_DIFF'] - rebounds_upd_1['AWAY_PLAYER_ID_5_RB_DIFF'].mean()) / rebounds_upd_1['AWAY_PLAYER_ID_5_RB_DIFF'].std()


In [112]:
heat_check = pd.DataFrame()

In [114]:
heat_check[['HOME_PLAYER_ID_1_PTS_ADJ', 'HOME_PLAYER_ID_2_PTS_ADJ', 'HOME_PLAYER_ID_3_PTS_ADJ', 'HOME_PLAYER_ID_4_PTS_ADJ', 'HOME_PLAYER_ID_5_PTS_ADJ', 'AWAY_PLAYER_ID_1_PTS_ADJ', 'AWAY_PLAYER_ID_2_PTS_ADJ', 'AWAY_PLAYER_ID_3_PTS_ADJ', 'AWAY_PLAYER_ID_4_PTS_ADJ', 'AWAY_PLAYER_ID_5_PTS_ADJ' ]] = points_upd[['HOME_PLAYER_ID_1_PTS_ADJ', 'HOME_PLAYER_ID_2_PTS_ADJ', 'HOME_PLAYER_ID_3_PTS_ADJ', 'HOME_PLAYER_ID_4_PTS_ADJ', 'HOME_PLAYER_ID_5_PTS_ADJ', 'AWAY_PLAYER_ID_1_PTS_ADJ', 'AWAY_PLAYER_ID_2_PTS_ADJ', 'AWAY_PLAYER_ID_3_PTS_ADJ', 'AWAY_PLAYER_ID_4_PTS_ADJ', 'AWAY_PLAYER_ID_5_PTS_ADJ' ]]

In [115]:
heat_check[['HOME_PLAYER_ID_1_RB_ADJ', 'HOME_PLAYER_ID_2_RB_ADJ',  'HOME_PLAYER_ID_3_RB_ADJ', 'HOME_PLAYER_ID_4_RB_ADJ', 'HOME_PLAYER_ID_5_RB_ADJ', 'AWAY_PLAYER_ID_1_RB_ADJ', 'AWAY_PLAYER_ID_2_RB_ADJ', 'AWAY_PLAYER_ID_3_RB_ADJ', 'AWAY_PLAYER_ID_4_RB_ADJ', 'AWAY_PLAYER_ID_5_RB_ADJ' ]] = rebounds_upd_1[['HOME_PLAYER_ID_1_RB_ADJ', 'HOME_PLAYER_ID_2_RB_ADJ',  'HOME_PLAYER_ID_3_RB_ADJ', 'HOME_PLAYER_ID_4_RB_ADJ', 'HOME_PLAYER_ID_5_RB_ADJ', 'AWAY_PLAYER_ID_1_RB_ADJ', 'AWAY_PLAYER_ID_2_RB_ADJ', 'AWAY_PLAYER_ID_3_RB_ADJ', 'AWAY_PLAYER_ID_4_RB_ADJ', 'AWAY_PLAYER_ID_5_RB_ADJ' ]]

In [116]:
heat_check.head()

,HOME_PLAYER_ID_1_PTS_ADJ,HOME_PLAYER_ID_2_PTS_ADJ,HOME_PLAYER_ID_3_PTS_ADJ,HOME_PLAYER_ID_4_PTS_ADJ,HOME_PLAYER_ID_5_PTS_ADJ,AWAY_PLAYER_ID_1_PTS_ADJ,AWAY_PLAYER_ID_2_PTS_ADJ,AWAY_PLAYER_ID_3_PTS_ADJ,AWAY_PLAYER_ID_4_PTS_ADJ,AWAY_PLAYER_ID_5_PTS_ADJ,HOME_PLAYER_ID_1_RB_ADJ,HOME_PLAYER_ID_2_RB_ADJ,HOME_PLAYER_ID_3_RB_ADJ,HOME_PLAYER_ID_4_RB_ADJ,HOME_PLAYER_ID_5_RB_ADJ,AWAY_PLAYER_ID_1_RB_ADJ,AWAY_PLAYER_ID_2_RB_ADJ,AWAY_PLAYER_ID_3_RB_ADJ,AWAY_PLAYER_ID_4_RB_ADJ,AWAY_PLAYER_ID_5_RB_ADJ
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.038355,-0.070744,-0.124484,-0.098484,-0.075150,-0.022092,-0.049774,-0.078082,-0.072918,-0.011921,-1.130127,-1.205388,-1.265294,-1.264930,-1.284997,-1.114430,-1.211840,-1.273991,-1.288510,-1.281342
2,-0.096198,-0.098225,-0.158921,-0.134635,-0.111643,-0.064360,-0.088484,-0.124025,-0.112606,-0.048347,-1.112583,-1.184145,-1.235478,-1.240875,-1.258910,-1.095460,-1.196815,-1.229364,-1.256376,-1.257562
3,-0.102287,-0.101117,-0.162546,-0.138440,-0.115484,-0.068810,-0.092559,-0.128861,-0.116784,-0.052181,-1.110737,-1.181909,-1.232340,-1.238343,-1.256164,-1.093463,-1.195234,-1.224667,-1.252993,-1.255059
4,-0.117508,-0.108349,-0.171609,-0.147953,-0.125088,-0.079933,-0.102746,-0.140951,-0.127228,-0.061767,-1.106120,-1.176318,-1.224494,-1.232013,-1.249299,-1.088471,-1.191280,-1.212923,-1.244537,-1.248801


In [117]:
heat_check[['SHOT_PLAYER_ID', 'SHOT_MADE',]] = df[['SHOT_PLAYER_ID', 'SHOT_MADE']]
heat_check[['HOME_PLAYER_ID_1', 'HOME_PLAYER_ID_2', 'HOME_PLAYER_ID_3', 'HOME_PLAYER_ID_4', 'HOME_PLAYER_ID_5', 'AWAY_PLAYER_ID_1', 'AWAY_PLAYER_ID_2', 'AWAY_PLAYER_ID_3', 'AWAY_PLAYER_ID_4', 'AWAY_PLAYER_ID_5']] = df[['HOME_PLAYER_ID_1', 'HOME_PLAYER_ID_2', 'HOME_PLAYER_ID_3', 'HOME_PLAYER_ID_4', 'HOME_PLAYER_ID_5', 'AWAY_PLAYER_ID_1', 'AWAY_PLAYER_ID_2', 'AWAY_PLAYER_ID_3', 'AWAY_PLAYER_ID_4', 'AWAY_PLAYER_ID_5']]

In [118]:
heat_check.head()

,HOME_PLAYER_ID_1_PTS_ADJ,HOME_PLAYER_ID_2_PTS_ADJ,HOME_PLAYER_ID_3_PTS_ADJ,HOME_PLAYER_ID_4_PTS_ADJ,HOME_PLAYER_ID_5_PTS_ADJ,AWAY_PLAYER_ID_1_PTS_ADJ,AWAY_PLAYER_ID_2_PTS_ADJ,AWAY_PLAYER_ID_3_PTS_ADJ,AWAY_PLAYER_ID_4_PTS_ADJ,AWAY_PLAYER_ID_5_PTS_ADJ,HOME_PLAYER_ID_1_RB_ADJ,HOME_PLAYER_ID_2_RB_ADJ,HOME_PLAYER_ID_3_RB_ADJ,HOME_PLAYER_ID_4_RB_ADJ,HOME_PLAYER_ID_5_RB_ADJ,AWAY_PLAYER_ID_1_RB_ADJ,AWAY_PLAYER_ID_2_RB_ADJ,AWAY_PLAYER_ID_3_RB_ADJ,AWAY_PLAYER_ID_4_RB_ADJ,AWAY_PLAYER_ID_5_RB_ADJ,SHOT_PLAYER_ID,SHOT_MADE,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115
1,-0.038355,-0.070744,-0.124484,-0.098484,-0.075150,-0.022092,-0.049774,-0.078082,-0.072918,-0.011921,-1.130127,-1.205388,-1.265294,-1.264930,-1.284997,-1.114430,-1.211840,-1.273991,-1.288510,-1.281342,NaN,NaN,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115
2,-0.096198,-0.098225,-0.158921,-0.134635,-0.111643,-0.064360,-0.088484,-0.124025,-0.112606,-0.048347,-1.112583,-1.184145,-1.235478,-1.240875,-1.258910,-1.095460,-1.196815,-1.229364,-1.256376,-1.257562,200794.0,False,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115
3,-0.102287,-0.101117,-0.162546,-0.138440,-0.115484,-0.068810,-0.092559,-0.128861,-0.116784,-0.052181,-1.110737,-1.181909,-1.232340,-1.238343,-1.256164,-1.093463,-1.195234,-1.224667,-1.252993,-1.255059,NaN,NaN,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115
4,-0.117508,-0.108349,-0.171609,-0.147953,-0.125088,-0.079933,-0.102746,-0.140951,-0.127228,-0.061767,-1.106120,-1.176318,-1.224494,-1.232013,-1.249299,-1.088471,-1.191280,-1.212923,-1.244537,-1.248801,201143.0,False,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115


In [119]:
heat_check.to_csv('/Users/seandiehl/Desktop/heat_check.csv')

In [120]:
assists = pd.DataFrame()
assists[['HOME_PLAYER_ID_1', 'HOME_PLAYER_ID_2', 'HOME_PLAYER_ID_3', 'HOME_PLAYER_ID_4', 'HOME_PLAYER_ID_5', 'AWAY_PLAYER_ID_1', 'AWAY_PLAYER_ID_2', 'AWAY_PLAYER_ID_3', 'AWAY_PLAYER_ID_4', 'AWAY_PLAYER_ID_5']] = df[['HOME_PLAYER_ID_1', 'HOME_PLAYER_ID_2', 'HOME_PLAYER_ID_3', 'HOME_PLAYER_ID_4', 'HOME_PLAYER_ID_5', 'AWAY_PLAYER_ID_1', 'AWAY_PLAYER_ID_2', 'AWAY_PLAYER_ID_3', 'AWAY_PLAYER_ID_4', 'AWAY_PLAYER_ID_5']]
assists[['ASSIST_PLAYER_ID', 'ASSIST_COUNT']] = df[['ASSIST_PLAYER_ID', 'ASSIST_COUNT']]

In [121]:
assists[[  'HOME_PLAYER_ID_1_EXP_AST',	'HOME_PLAYER_ID_2_EXP_AST',	'HOME_PLAYER_ID_3_EXP_AST',	'HOME_PLAYER_ID_4_EXP_AST',	'HOME_PLAYER_ID_5_EXP_AST',	'AWAY_PLAYER_ID_1_EXP_AST',	'AWAY_PLAYER_ID_2_EXP_AST',	'AWAY_PLAYER_ID_3_EXP_AST',	'AWAY_PLAYER_ID_4_EXP_AST',	'AWAY_PLAYER_ID_5_EXP_AST' ]] = df[[  'HOME_PLAYER_ID_1_EXP_AST',	'HOME_PLAYER_ID_2_EXP_AST',	'HOME_PLAYER_ID_3_EXP_AST',	'HOME_PLAYER_ID_4_EXP_AST',	'HOME_PLAYER_ID_5_EXP_AST',	'AWAY_PLAYER_ID_1_EXP_AST',	'AWAY_PLAYER_ID_2_EXP_AST',	'AWAY_PLAYER_ID_3_EXP_AST',	'AWAY_PLAYER_ID_4_EXP_AST',	'AWAY_PLAYER_ID_5_EXP_AST' ]] = df[[  'HOME_PLAYER_ID_1_EXP_AST',	'HOME_PLAYER_ID_2_EXP_AST',	'HOME_PLAYER_ID_3_EXP_AST',	'HOME_PLAYER_ID_4_EXP_AST',	'HOME_PLAYER_ID_5_EXP_AST',	'AWAY_PLAYER_ID_1_EXP_AST',	'AWAY_PLAYER_ID_2_EXP_AST',	'AWAY_PLAYER_ID_3_EXP_AST',	'AWAY_PLAYER_ID_4_EXP_AST',	'AWAY_PLAYER_ID_5_EXP_AST' ]]

In [122]:
assists.to_csv('/Users/seandiehl/Desktop/assists.csv')

In [124]:
assists_players = pd.read_csv('/Users/seandiehl/Desktop/assists_players.csv')
assists_players['TIME'] = df['TIME']

In [125]:
assists_players.head()

,Unnamed: 0,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5,ASSIST_PLAYER_ID,ASSIST_COUNT,HOME_PLAYER_ID_1_EXP_AST,HOME_PLAYER_ID_2_EXP_AST,HOME_PLAYER_ID_3_EXP_AST,HOME_PLAYER_ID_4_EXP_AST,HOME_PLAYER_ID_5_EXP_AST,AWAY_PLAYER_ID_1_EXP_AST,AWAY_PLAYER_ID_2_EXP_AST,AWAY_PLAYER_ID_3_EXP_AST,AWAY_PLAYER_ID_4_EXP_AST,AWAY_PLAYER_ID_5_EXP_AST,203518,203112,203500,1628389,200746,1628386,1628443,202329,1626147,203937,201583,1628387,203507,2546,1628384,1627853,2772,201571,1628407,1627735,1628366,203084,203115,201587,203382,203145,203078,1627736,201563,1628395,1627761,1627733,202722,201976,203920,1628444,202687,202357,1628469,202339,203992,202711,1626164,1628449,202340,1628396,1627763,1628415,202705,1627759,203485,1628425,1628418,203493,203504,202692,202710,203998,203484,203477,203991,201960,203487,1627936,1626161,1628463,201163,1627737,203546,203903,1628499,1628381,1628380,201954,201144,1626192,1626188,202326,203496,203459,1628470,203109,201967,201939,203584,203076,1627738,202334,203473,1626155,203521,201942,1627767,203476,203915,1628416,1628422,1628408,201609,203083,1627739,201142,1627740,201961,203954,203516,1628393,201936,203957,202702,202324,1626245,1628390,1627812,1627827,1627854,203095,1628368,204025,202683,1628365,201568,204038,201188,200752,202331,201959,203497,203932,201569,1626203,203924,1626170,1627982,201980,203110,203210,201145,201933,1627772,203501,201935,203090,1626149,203914,203925,202699,1626150,1627885,1628404,2617,202330,203089,1627823,1626195,1626209,1628439,1627741,201588,203524,201950,203200,204066,1626178,1626158,203918,203816,201143,1627863,2730,1626154,201586,2738,101141,204060,201281,1627742,202681,1628371,1628411,1627743,1628367,1628382,202704,2544,2229,201973,101161,201949,2207,1626169,204020,202325,203999,1627745,1627884,1627883,1626145,201599,202709,1626163,1628379,203077,1628467,1627788,1628436,201585,1628398,1627746,203087,203897,1627774,1628388,201584,1627814,1627215,203458,202695,203086,1627747,203081,202391,1626172,201572,201577,201567,200768,1627789,1628399,1626168,202714,203705,1627748,1626246,1628374,1626185,1628412,202083,201601,1627775,203468,204456,203926,201580,203585,1628035,203463,201975,1626257,203114,101139,203121,1626259,201988,200794,202703,1628378,1628513,1628370,202328,202734,203961,1628537,202694,202693,1628420,1628405,1626144,1627749,1627750,203488,1627846,1626204,203894,203526,1627777,201149,203457,1628373,203994,1628021,1626220,203124,1628400,1626143,203506,203482,1626224,1626162,203953,202335,1628383,101108,1626166,203901,1627780,1628409,203486,203101,1627751,203490,1626171,204001,203939,1626181,1627816,1627752,1627753,1628397,203944,1628432,1626196,1627781,203085,203460,1628421,203922,200765,201565,203082,1626179,201937,1626156,1627734,203960,203967,203107,203471,203118,1627782,202697,1627783,1627732,203613,1628424,203935,1628372,202397,203503,203917,202362,1628410,1628403,1628369,201952,202066,1628462,1628464,202738,202498,202691,202684,1628414,201229,1626157,200782,1626167,202327,1627755,202685,1627756,1627832,203943,202696,203079,202689,202322,1627820,1628476,203933,201566,1628401,1627855,202355,203952,1626210,203710,101150,1627786,1628391,1626159,1626153,1627824,201152,203469,203092,1627790,1627826,TIME
0,0,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,NaN,NaN,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [128]:
def apply_assists(player_id):
   
    
    # note 1
    assists_players.loc[(assists_players['TIME']==0), str(player_id)] = 0

    assists_players.loc[(assists_players['ASSIST_PLAYER_ID']==player_id), str(player_id)] = assists_players['ASSIST_COUNT']

    assists_players[str(player_id)] = assists_players[str(player_id)].ffill()

In [134]:
for column in assists_players.columns[23:-1]:
    player_id = int(column)
    apply_assists(player_id)

In [137]:
assists_players.to_csv('/Users/seandiehl/Desktop/assists_players_upd.csv')

In [139]:
assist_players_upd = pd.read_csv('/Users/seandiehl/Desktop/assists_players_upd.csv')

In [140]:
assist_players_upd.head()

,Unnamed: 0.1,Unnamed: 0,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5,ASSIST_PLAYER_ID,ASSIST_COUNT,HOME_PLAYER_ID_1_EXP_AST,HOME_PLAYER_ID_2_EXP_AST,HOME_PLAYER_ID_3_EXP_AST,HOME_PLAYER_ID_4_EXP_AST,HOME_PLAYER_ID_5_EXP_AST,AWAY_PLAYER_ID_1_EXP_AST,AWAY_PLAYER_ID_2_EXP_AST,AWAY_PLAYER_ID_3_EXP_AST,AWAY_PLAYER_ID_4_EXP_AST,AWAY_PLAYER_ID_5_EXP_AST,HOME_PLAYER_ID_1_ACT_AST,HOME_PLAYER_ID_2_ACT_AST,HOME_PLAYER_ID_3_ACT_AST,HOME_PLAYER_ID_4_ACT_AST,HOME_PLAYER_ID_5_ACT_AST,AWAY_PLAYER_ID_1_ACT_AST,AWAY_PLAYER_ID_2_ACT_AST,AWAY_PLAYER_ID_3_ACT_AST,AWAY_PLAYER_ID_4_ACT_AST,AWAY_PLAYER_ID_5_ACT_AST,203518,203112,203500,1628389,200746,1628386,1628443,202329,1626147,203937,201583,1628387,203507,2546,1628384,1627853,2772,201571,1628407,1627735,1628366,203084,203115,201587,203382,203145,203078,1627736,201563,1628395,1627761,1627733,202722,201976,203920,1628444,202687,202357,1628469,202339,203992,202711,1626164,1628449,202340,1628396,1627763,1628415,202705,1627759,203485,1628425,1628418,203493,203504,202692,202710,203998,203484,203477,203991,201960,203487,1627936,1626161,1628463,201163,1627737,203546,203903,1628499,1628381,1628380,201954,201144,1626192,1626188,202326,203496,203459,1628470,203109,201967,201939,203584,203076,1627738,202334,203473,1626155,203521,201942,1627767,203476,203915,1628416,1628422,1628408,201609,203083,1627739,201142,1627740,201961,203954,203516,1628393,201936,203957,202702,202324,1626245,1628390,1627812,1627827,1627854,203095,1628368,204025,202683,1628365,201568,204038,201188,200752,202331,201959,203497,203932,201569,1626203,203924,1626170,1627982,201980,203110,203210,201145,201933,1627772,203501,201935,203090,1626149,203914,203925,202699,1626150,1627885,1628404,2617,202330,203089,1627823,1626195,1626209,1628439,1627741,201588,203524,201950,203200,204066,1626178,1626158,203918,203816,201143,1627863,2730,1626154,201586,2738,101141,204060,201281,1627742,202681,1628371,1628411,1627743,1628367,1628382,202704,2544,2229,201973,101161,201949,2207,1626169,204020,202325,203999,1627745,1627884,1627883,1626145,201599,202709,1626163,1628379,203077,1628467,1627788,1628436,201585,1628398,1627746,203087,203897,1627774,1628388,201584,1627814,1627215,203458,202695,203086,1627747,203081,202391,1626172,201572,201577,201567,200768,1627789,1628399,1626168,202714,203705,1627748,1626246,1628374,1626185,1628412,202083,201601,1627775,203468,204456,203926,201580,203585,1628035,203463,201975,1626257,203114,101139,203121,1626259,201988,200794,202703,1628378,1628513,1628370,202328,202734,203961,1628537,202694,202693,1628420,1628405,1626144,1627749,1627750,203488,1627846,1626204,203894,203526,1627777,201149,203457,1628373,203994,1628021,1626220,203124,1628400,1626143,203506,203482,1626224,1626162,203953,202335,1628383,101108,1626166,203901,1627780,1628409,203486,203101,1627751,203490,1626171,204001,203939,1626181,1627816,1627752,1627753,1628397,203944,1628432,1626196,1627781,203085,203460,1628421,203922,200765,201565,203082,1626179,201937,1626156,1627734,203960,203967,203107,203471,203118,1627782,202697,1627783,1627732,203613,1628424,203935,1628372,202397,203503,203917,202362,1628410,1628403,1628369,201952,202066,1628462,1628464,202738,202498,202691,202684,1628414,201229,1626157,200782,1626167,202327,1627755,202685,1627756,1627832,203943,202696,203079,202689,202322,1627820,1628476,203933,201566,1628401,1627855,202355,203952,1626210,203710,101150,1627786,1628391,1626159,1626153,1627824,201152,203469,203092,1627790,1627826,TIME
0,0,0,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,NaN,NaN,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,#VALUE!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [141]:
assist_players_upd = assist_players_upd.replace(to_replace="#VALUE!", value=np.nan)
assist_players_upd = assist_players_upd.astype(float)

In [142]:
assist_players_upd['HOME_PLAYER_ID_1_AST_DIFF'] = assist_players_upd['HOME_PLAYER_ID_1_ACT_AST'] - assist_players_upd['HOME_PLAYER_ID_1_EXP_AST']
assist_players_upd['HOME_PLAYER_ID_2_AST_DIFF'] = assist_players_upd['HOME_PLAYER_ID_2_ACT_AST'] - assist_players_upd['HOME_PLAYER_ID_2_EXP_AST']
assist_players_upd['HOME_PLAYER_ID_3_AST_DIFF'] = assist_players_upd['HOME_PLAYER_ID_3_ACT_AST'] - assist_players_upd['HOME_PLAYER_ID_3_EXP_AST']
assist_players_upd['HOME_PLAYER_ID_4_AST_DIFF'] = assist_players_upd['HOME_PLAYER_ID_4_ACT_AST'] - assist_players_upd['HOME_PLAYER_ID_4_EXP_AST']
assist_players_upd['HOME_PLAYER_ID_5_AST_DIFF'] = assist_players_upd['HOME_PLAYER_ID_5_ACT_AST'] - assist_players_upd['HOME_PLAYER_ID_5_EXP_AST']

assist_players_upd['AWAY_PLAYER_ID_1_AST_DIFF'] = assist_players_upd['AWAY_PLAYER_ID_1_ACT_AST'] - assist_players_upd['AWAY_PLAYER_ID_1_EXP_AST']
assist_players_upd['AWAY_PLAYER_ID_2_AST_DIFF'] = assist_players_upd['AWAY_PLAYER_ID_2_ACT_AST'] - assist_players_upd['AWAY_PLAYER_ID_2_EXP_AST']
assist_players_upd['AWAY_PLAYER_ID_3_AST_DIFF'] = assist_players_upd['AWAY_PLAYER_ID_3_ACT_AST'] - assist_players_upd['AWAY_PLAYER_ID_3_EXP_AST']
assist_players_upd['AWAY_PLAYER_ID_4_AST_DIFF'] = assist_players_upd['AWAY_PLAYER_ID_4_ACT_AST'] - assist_players_upd['AWAY_PLAYER_ID_4_EXP_AST']
assist_players_upd['AWAY_PLAYER_ID_5_AST_DIFF'] = assist_players_upd['AWAY_PLAYER_ID_5_ACT_AST'] - assist_players_upd['AWAY_PLAYER_ID_5_EXP_AST']

In [145]:
assist_players_upd['HOME_PLAYER_ID_1_AST_ADJ'] = (assist_players_upd['HOME_PLAYER_ID_1_AST_DIFF'] - assist_players_upd['HOME_PLAYER_ID_1_AST_DIFF'].mean()) / assist_players_upd['HOME_PLAYER_ID_1_AST_DIFF'].std()
assist_players_upd['HOME_PLAYER_ID_1_AST_ADJ'] = (assist_players_upd['HOME_PLAYER_ID_2_AST_DIFF'] - assist_players_upd['HOME_PLAYER_ID_2_AST_DIFF'].mean()) / assist_players_upd['HOME_PLAYER_ID_2_AST_DIFF'].std()
assist_players_upd['HOME_PLAYER_ID_3_AST_ADJ'] = (assist_players_upd['HOME_PLAYER_ID_3_AST_DIFF'] - assist_players_upd['HOME_PLAYER_ID_3_AST_DIFF'].mean()) / assist_players_upd['HOME_PLAYER_ID_3_AST_DIFF'].std()
assist_players_upd['HOME_PLAYER_ID_4_AST_ADJ'] = (assist_players_upd['HOME_PLAYER_ID_4_AST_DIFF'] - assist_players_upd['HOME_PLAYER_ID_4_AST_DIFF'].mean()) / assist_players_upd['HOME_PLAYER_ID_4_AST_DIFF'].std()
assist_players_upd['HOME_PLAYER_ID_5_AST_ADJ'] = (assist_players_upd['HOME_PLAYER_ID_5_AST_DIFF'] - assist_players_upd['HOME_PLAYER_ID_5_AST_DIFF'].mean()) / assist_players_upd['HOME_PLAYER_ID_5_AST_DIFF'].std()

assist_players_upd['AWAY_PLAYER_ID_1_AST_ADJ'] = (assist_players_upd['AWAY_PLAYER_ID_1_AST_DIFF'] - assist_players_upd['AWAY_PLAYER_ID_1_AST_DIFF'].mean()) / assist_players_upd['AWAY_PLAYER_ID_1_AST_DIFF'].std()
assist_players_upd['AWAY_PLAYER_ID_2_AST_ADJ'] = (assist_players_upd['AWAY_PLAYER_ID_2_AST_DIFF'] - assist_players_upd['AWAY_PLAYER_ID_2_AST_DIFF'].mean()) / assist_players_upd['AWAY_PLAYER_ID_2_AST_DIFF'].std()
assist_players_upd['AWAY_PLAYER_ID_3_AST_ADJ'] = (assist_players_upd['AWAY_PLAYER_ID_3_AST_DIFF'] - assist_players_upd['AWAY_PLAYER_ID_3_AST_DIFF'].mean()) / assist_players_upd['AWAY_PLAYER_ID_3_AST_DIFF'].std()
assist_players_upd['AWAY_PLAYER_ID_4_AST_ADJ'] = (assist_players_upd['AWAY_PLAYER_ID_4_AST_DIFF'] - assist_players_upd['AWAY_PLAYER_ID_4_AST_DIFF'].mean()) / assist_players_upd['AWAY_PLAYER_ID_4_AST_DIFF'].std()
assist_players_upd['AWAY_PLAYER_ID_5_AST_ADJ'] = (assist_players_upd['AWAY_PLAYER_ID_5_AST_DIFF'] - assist_players_upd['AWAY_PLAYER_ID_5_AST_DIFF'].mean()) / assist_players_upd['AWAY_PLAYER_ID_5_AST_DIFF'].std()



In [146]:
heat_check[['HOME_PLAYER_ID_1_AST_ADJ', 'HOME_PLAYER_ID_2_AST_ADJ', 'HOME_PLAYER_ID_3_AST_ADJ', 'HOME_PLAYER_ID_4_AST_ADJ', 'HOME_PLAYER_ID_5_AST_ADJ', 'AWAY_PLAYER_ID_1_AST_ADJ', 'AWAY_PLAYER_ID_2_AST_ADJ', 'AWAY_PLAYER_ID_3_AST_ADJ', 'AWAY_PLAYER_ID_4_AST_ADJ', 'AWAY_PLAYER_ID_5_AST_ADJ']] = assist_players_upd[['HOME_PLAYER_ID_1_AST_ADJ', 'HOME_PLAYER_ID_2_AST_ADJ', 'HOME_PLAYER_ID_3_AST_ADJ', 'HOME_PLAYER_ID_4_AST_ADJ', 'HOME_PLAYER_ID_5_AST_ADJ', 'AWAY_PLAYER_ID_1_AST_ADJ', 'AWAY_PLAYER_ID_2_AST_ADJ', 'AWAY_PLAYER_ID_3_AST_ADJ', 'AWAY_PLAYER_ID_4_AST_ADJ', 'AWAY_PLAYER_ID_5_AST_ADJ']]

In [147]:
heat_check.head()

,HOME_PLAYER_ID_1_PTS_ADJ,HOME_PLAYER_ID_2_PTS_ADJ,HOME_PLAYER_ID_3_PTS_ADJ,HOME_PLAYER_ID_4_PTS_ADJ,HOME_PLAYER_ID_5_PTS_ADJ,AWAY_PLAYER_ID_1_PTS_ADJ,AWAY_PLAYER_ID_2_PTS_ADJ,AWAY_PLAYER_ID_3_PTS_ADJ,AWAY_PLAYER_ID_4_PTS_ADJ,AWAY_PLAYER_ID_5_PTS_ADJ,HOME_PLAYER_ID_1_RB_ADJ,HOME_PLAYER_ID_2_RB_ADJ,HOME_PLAYER_ID_3_RB_ADJ,HOME_PLAYER_ID_4_RB_ADJ,HOME_PLAYER_ID_5_RB_ADJ,AWAY_PLAYER_ID_1_RB_ADJ,AWAY_PLAYER_ID_2_RB_ADJ,AWAY_PLAYER_ID_3_RB_ADJ,AWAY_PLAYER_ID_4_RB_ADJ,AWAY_PLAYER_ID_5_RB_ADJ,SHOT_PLAYER_ID,SHOT_MADE,HOME_PLAYER_ID_1,HOME_PLAYER_ID_2,HOME_PLAYER_ID_3,HOME_PLAYER_ID_4,HOME_PLAYER_ID_5,AWAY_PLAYER_ID_1,AWAY_PLAYER_ID_2,AWAY_PLAYER_ID_3,AWAY_PLAYER_ID_4,AWAY_PLAYER_ID_5,HOME_PLAYER_ID_1_AST_ADJ,HOME_PLAYER_ID_2_AST_ADJ,HOME_PLAYER_ID_3_AST_ADJ,HOME_PLAYER_ID_4_AST_ADJ,HOME_PLAYER_ID_5_AST_ADJ,AWAY_PLAYER_ID_1_AST_ADJ,AWAY_PLAYER_ID_2_AST_ADJ,AWAY_PLAYER_ID_3_AST_ADJ,AWAY_PLAYER_ID_4_AST_ADJ,AWAY_PLAYER_ID_5_AST_ADJ
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.038355,-0.070744,-0.124484,-0.098484,-0.075150,-0.022092,-0.049774,-0.078082,-0.072918,-0.011921,-1.130127,-1.205388,-1.265294,-1.264930,-1.284997,-1.114430,-1.211840,-1.273991,-1.288510,-1.281342,NaN,NaN,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,-0.170090,-0.135935,-0.141344,-0.137509,-0.131445,-0.124255,-0.060869,-0.104121,-0.093769,-0.083862
2,-0.096198,-0.098225,-0.158921,-0.134635,-0.111643,-0.064360,-0.088484,-0.124025,-0.112606,-0.048347,-1.112583,-1.184145,-1.235478,-1.240875,-1.258910,-1.095460,-1.196815,-1.229364,-1.256376,-1.257562,200794.0,False,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,-0.202590,-0.170460,-0.175962,-0.151126,-0.144571,-0.147033,-0.081385,-0.148083,-0.117589,-0.111963
3,-0.102287,-0.101117,-0.162546,-0.138440,-0.115484,-0.068810,-0.092559,-0.128861,-0.116784,-0.052181,-1.110737,-1.181909,-1.232340,-1.238343,-1.256164,-1.093463,-1.195234,-1.224667,-1.252993,-1.255059,NaN,NaN,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,-0.206011,-0.174094,-0.179606,-0.152560,-0.145953,-0.149431,-0.083544,-0.152711,-0.120096,-0.114921
4,-0.117508,-0.108349,-0.171609,-0.147953,-0.125088,-0.079933,-0.102746,-0.140951,-0.127228,-0.061767,-1.106120,-1.176318,-1.224494,-1.232013,-1.249299,-1.088471,-1.191280,-1.212923,-1.244537,-1.248801,201143.0,False,202681,203935,201143,202694,1628369,1627750,203914,203999,200794,203115,-0.214564,-0.183180,-0.188716,-0.156143,-0.149407,-0.155426,-0.088943,-0.164279,-0.126365,-0.122317
